<a href="https://colab.research.google.com/github/hariPrasadCoder/10-pointer-analysis/blob/main/10_pointer_for_all_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import numpy as np

In [2]:
df = pd.read_csv('S&P 500.csv')

In [3]:
df

,Symbol,Company Name,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",8/9/1976,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",10/6/1997,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",12/23/2019,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",8/7/2001,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",6/22/2001,109380,1873


In [4]:
company_names = []
for i in df['Company Name']:
  cleaned = i.lower().replace(' ','_')
  company_names.append(cleaned)
df['Company Name'] = company_names
print(company_names)

['3m', 'abbott_laboratories', 'abbvie', 'abiomed', 'accenture', 'activision_blizzard', 'adobe', 'advanced_micro_devices', 'advance_auto_parts', 'aes_corp', 'aflac', 'agilent_technologies', 'air_products_&_chemicals', 'akamai_technologies', 'alaska_air_group', 'albemarle_corporation', 'alexandria_real_estate_equities', 'align_technology', 'allegion', 'alliant_energy', 'allstate_corp', 'alphabet\xa0(class_a)', 'alphabet\xa0(class_c)', 'altria_group', 'amazon', 'amcor', 'ameren_corp', 'american_airlines_group', 'american_electric_power', 'american_express', 'american_international_group', 'american_tower', 'american_water_works', 'ameriprise_financial', 'amerisourcebergen', 'ametek', 'amgen', 'amphenol', 'analog_devices', 'ansys', 'anthem', 'aon', 'a._o._smith', 'apa_corporation', 'apple', 'applied_materials', 'aptiv', 'adm', 'arista_networks', 'arthur_j._gallagher_&_co.', 'assurant', 'at&t', 'atmos_energy', 'autodesk', 'automatic_data_processing', 'autozone', 'avalonbay_communities', 'av

In [5]:
tickers = df['Symbol']
print(tickers)

0       MMM
1       ABT
2      ABBV
3      ABMD
4       ACN
       ... 
500     YUM
501    ZBRA
502     ZBH
503    ZION
504     ZTS
Name: Symbol, Length: 505, dtype: object


In [6]:
tabs = []
count = 0
for i in range(len(tickers)-1):
  raw_url = 'https://www.macrotrends.net/stocks/charts'
  company_url = raw_url + '/' + tickers[count] + '/' + company_names[count] + '/'
  count += 1

  # print(i)
  # print(company_names[count])
  try:
    #pe-ratio
    pe_page = requests.get(company_url + 'pe-ratio')
    pe_soup = bs(pe_page.content,"lxml")
    pe_ratio = float(pe_soup.find("table").find_all("tr")[2].find_all("td")[-1].text)

    #current ratio
    cr_page = requests.get(company_url +'current-ratio')
    cr_soup = bs(cr_page.content,'lxml')
    current_ratio = float(cr_soup.find_all('tr')[2].find_all('td')[-1].text)

    #roce
    ebit_page = requests.get(company_url + 'ebit')
    ebit_soup = bs(ebit_page.content)
    ebit = int(ebit_soup.find('table',class_='historical_data_table table').find_all('tr')[1].find_all('td')[1].text.replace(',','')[1:])

    ltl_page = requests.get(company_url +'total-long-term-liabilities')
    ltl_soup = bs(ltl_page.content)
    long_term_liabilities = int(ltl_soup.find('table',class_='historical_data_table table').find_all('tr')[1].find_all('td')[1].text.replace(',','')[1:])

    she_page = requests.get(company_url +'total-share-holder-equity')
    she_soup = bs(she_page.content)
    share_holders_equity = int(she_soup.find('table',class_='historical_data_table table').find_all('tr')[1].find_all('td')[1].text.replace(',','')[1:])

    roce = ebit/(long_term_liabilities + share_holders_equity)

    #Debt to Equity
    d2e_page = requests.get(company_url +'debt-equity-ratio')
    d2e_soup = bs(d2e_page.content,'lxml')
    debt2equity = float(d2e_soup.find_all('tr')[2].find_all('td')[-1].text)

    #Inventory Turnover Ratio
    itr_page = requests.get(company_url +'inventory-turnover')
    itr_soup = bs(itr_page.content,'lxml')
    inventory_turnover = float(itr_soup.find(id = 'main_content').find('strong').text)

    #Free Cash Flow
    fcf_page = requests.get(company_url +'free-cash-flow')
    fcf_soup = bs(fcf_page.content)
    free_cash_flow = float(fcf_soup.find('table',class_='historical_data_table table').find_all('tr')[1].find_all('td')[1].text.replace(',',''))

    #Increase in Reserves trend
    def trendline(index,data, order=1):
      coeffs = np.polyfit(index, list(data), order)
      slope = coeffs[-2]
      return float(slope)
    
    rt_page = requests.get(company_url +'retained-earnings-accumulated-deficit')
    rt_soup = bs(rt_page.content)
    reserve_list = []
    for i in range(1,6):
      reserve = int(rt_soup.find('table',class_='historical_data_table table').find_all('tr')[i].find_all('td')[1].text.replace(',','')[1:])
      reserve_list.append(reserve)

    reserve_list = reserve_list[::-1]

    index=[1,2,3,4,5]
    reserves_trend = trendline(index,reserve_list)

    #Quick Ratio
    qr_page = requests.get(company_url +'quick-ratio')
    qr_soup = bs(qr_page.content)
    quick_ratio = float(qr_soup.find('table').find_all('tr')[2].find_all('td')[-1].text)

    #Return on Equity Ratio
    roe_page = requests.get(company_url +'roe')
    roe_soup = bs(roe_page.content)
    roe = float(roe_soup.find('table').find_all('tr')[2].find_all('td')[-1].text[:-1]) /100

    #Return on Investment
    roi_page = requests.get(company_url +'roi')
    roi_soup = bs(roi_page.content)
    roi = float(roi_soup.find('table').find_all('tr')[2].find_all('td')[-1].text[:-1]) /100
    
    print(tickers[count] + ' ' + company_names[count] + ' is scraped')

    tabs.append({
        'ticker': tickers[count],
        'company_name': company_names[count],
        'pe_ratio': pe_ratio,
        'current_ratio': current_ratio,
        'roce': roce,
        'debt_to_equity_ratio': debt2equity,
        'inventory_turnover_ratio': inventory_turnover,
        'free_cash_flow': free_cash_flow,
        'reserves_trend': reserves_trend,
        'quick_ratio': quick_ratio,
        'return_on_equity_ratio': roe,
        'return_on_investment': roi
    })
  

  except:
    print('There is an error with '+ tickers[count] + ' ' + company_names[count])
  

df1 = pd.DataFrame(tabs)

ABT abbott_laboratories is scraped
ABBV abbvie is scraped
ABMD abiomed is scraped
ACN accenture is scraped
There is an error with ATVI activision_blizzard
There is an error with ADBE adobe
There is an error with AMD advanced_micro_devices
AAP advance_auto_parts is scraped
AES aes_corp is scraped
AFL aflac is scraped
There is an error with A agilent_technologies
APD air_products_&_chemicals is scraped
AKAM akamai_technologies is scraped
There is an error with ALK alaska_air_group
There is an error with ALB albemarle_corporation
ARE alexandria_real_estate_equities is scraped
There is an error with ALGN align_technology
ALLE allegion is scraped
LNT alliant_energy is scraped
ALL allstate_corp is scraped
There is an error with GOOGL alphabet (class_a)
GOOG alphabet (class_c) is scraped
MO altria_group is scraped
AMZN amazon is scraped
AMCR amcor is scraped
AEE ameren_corp is scraped
AAL american_airlines_group is scraped
AEP american_electric_power is scraped
AXP american_express is scraped

In [7]:
df1.to_csv("scraped_data.csv")

In [8]:
df1

,ticker,company_name,pe_ratio,current_ratio,roce,debt_to_equity_ratio,inventory_turnover_ratio,free_cash_flow,reserves_trend,quick_ratio,return_on_equity_ratio,return_on_investment
0,ABT,abbott_laboratories,17.43,1.71,0.181770,2.32,0.98,6740.00,1472.8,1.19,0.4245,0.2471
1,ABBV,abbvie,34.92,1.83,0.088340,1.13,0.84,5724.00,599.3,1.42,0.2139,0.1643
2,ABMD,abiomed,32.24,1.02,0.093213,9.96,1.42,16790.00,-738.8,0.91,0.5686,0.1881
3,ACN,accenture,110.85,7.64,0.168375,0.11,0.51,221.20,221.2,6.89,0.1128,inf
4,AAP,advance_auto_parts,44.55,2.24,0.209167,0.56,1.17,777.00,707.6,1.71,0.5988,0.4384
...,...,...,...,...,...,...,...,...,...,...,...,...
317,XLNX,xilinx,23.18,0.78,0.042560,2.79,2.23,-2521.00,497.2,0.68,0.1060,0.0615
318,XYL,xylem,66.05,6.81,0.153830,0.77,1.06,1043.56,-77.2,6.35,0.2666,0.1852
319,YUM,yum!_brands,47.03,2.33,0.054018,1.63,1.17,641.00,243.3,1.81,0.1531,0.1138
320,ZBH,zimmer_biomet,37.96,0.96,0.184733,1.08,1.80,895.00,397.6,0.69,0.3394,0.2886
